In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
gene_x_sample = pd.read_csv(PATH["gene_x_sample.tsv"], sep="\t", index_col=0)

target_x_sample = pd.read_csv(PATH["target_x_sample.tsv"], sep="\t", index_col=0)

gene_sets = ccal.read_gmts(SETTING["gene_set_file_paths"])

gene_set_x_sample = pd.read_csv(PATH["gene_set_x_sample.tsv"], sep="\t", index_col=0)

In [ ]:
for target_name, target_values in target_x_sample.iterrows():

    output_directory_path = "{}/{}".format(PATH["expand_gene_set/"], target_name)

    ccal.establish_path(output_directory_path, "directory")

    target_values = target_values[target_values != -1]

    score_moe_p_value_fdr = pd.concat(
        (
            pd.read_csv(
                "{}/{}/all.tsv".format(
                    PATH["find_differentially_expressed_gene/"], target_name
                ),
                sep="\t",
                index_col=0,
            ),
            pd.read_csv(
                "{}/{}/all.tsv".format(
                    PATH["find_differentially_expressed_gene_set/"], target_name
                ),
                sep="\t",
                index_col=0,
            ),
        )
    )

    for gene_set_name in SETTING["gene_sets_to_peek"]:

        features = pd.concat(
            (
                gene_x_sample.reindex(gene_sets.loc[gene_set_name].dropna()),
                gene_set_x_sample.reindex((gene_set_name,)),
            )
        )

        ccal.make_match_panel(
            target_values,
            features,
            score_moe_p_value_fdr=score_moe_p_value_fdr,
            n_extreme=None,
            target_type="binary",
            plot_std=SETTING["plot_std"],
            title=gene_set_name,
            file_path_prefix="{}/{}".format(output_directory_path, gene_set_name),
        )